In [1]:
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import calculations.item_functions as feature
import pandas as pd


In [2]:
data = pd.read_csv('../datasets/custom_brfss_full.csv')
columns = ['Diabetes', 'Sex', 'CurrentSmoker', 'SmokedMin100', 'CurrentDrinker', 'Exercise', 'High_Blood_Pressure', 'Weight_Category', 'General_Health']
data = data.dropna(subset=columns) #delete rows with NaN values in any of the used columns

# sample 10000 rows, with 5000 rows for each class
data = data.groupby('Diabetes').apply(lambda x: x.sample(n=5000, random_state=0))
data = data.reset_index(drop=True)

y = data['Diabetes'].copy()
y = y.replace({'No': 0, 'Yes': 1})
X = data.drop(['Diabetes', 'Stroke', 'HeartAttack', 'SkinCancer', 'Asthma', 'Angina_or_Coronary_Heart_Disease', 'Arthritis', 'KidneyDisease', 'Diabetes', 'Smoker'], axis=1)
X['Sex'] = pd.to_numeric(X['Sex'].replace({'Female': 1, 'Male': 0}))
X['CurrentSmoker'] = pd.to_numeric(X['CurrentSmoker'].replace({'No': 0, 'Yes': 1}))
X['SmokedMin100'] = pd.to_numeric(X['SmokedMin100'].replace({'No': 0, 'Yes': 1}))
X['CurrentDrinker'] = pd.to_numeric(X['CurrentDrinker'].replace({'No': 0, 'Yes': 1}))
X['Exercise'] = pd.to_numeric(X['Exercise'].replace({'No': 0, 'Yes': 1}))
X['High_Blood_Pressure'] = pd.to_numeric(X['High_Blood_Pressure'].replace({'No': 0, 'Yes': 1}))
X['Weight_Category'] = pd.to_numeric(X['Weight_Category'].replace({'Underweight': 0, 'Normal Weight': 1, 'Overweight': 2, 'Obese': 3}))
X['General_Health'] = pd.to_numeric(X['General_Health'].replace({'Excellent': 0, 'Very good': 1, 'Good': 2, 'Fair': 3, 'Poor': 4}))
X

C:\Users\Anna\AppData\Local\Temp\ipykernel_1272\2522028692.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('Diabetes').apply(lambda x: x.sample(n=5000, random_state=0))
C:\Users\Anna\AppData\Local\Temp\ipykernel_1272\2522028692.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'No': 0, 'Yes': 1})
C:\Users\Anna\AppData\Local\Temp\ipykernel_1272\2522028692.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and 

,BMI,Age,Sex,CurrentSmoker,SmokedMin100,AlcPerWeek,CurrentDrinker,Exercise,General_Health,High_Blood_Pressure,Fruit_Per_Day,Vegetable_Per_Day,Weight_Category
0,30.0,63,0,0,0,0.0,0,1,1,1,0.02,0.02,2
1,32.0,66,0,0,1,0.0,0,1,3,0,1.00,3.30,3
2,31.0,60,1,1,1,0.0,0,0,2,1,0.17,1.00,3
3,39.0,42,0,0,0,653.0,1,1,2,0,1.00,1.00,3
4,27.0,51,0,1,1,70.0,1,1,3,0,0.00,0.29,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,27.0,61,1,1,1,0.0,0,1,1,0,0.29,0.71,2
9996,31.0,74,1,0,0,0.0,0,1,2,1,1.00,0.14,3
9997,42.0,66,1,0,0,1000.0,1,0,2,1,0.71,0.71,3
9998,26.0,55,0,0,0,47.0,1,1,0,0,1.00,1.00,2


In [3]:
X = X.select_dtypes(include=[np.number]) #filter X to only include numeric columns
X = X.fillna(X.mean()) #fill NaN values with the mean of the column


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

X_train_means = X_train.mean().to_frame().T

COLUMNS = X_train.columns

nn = DecisionTreeClassifier()
nn.fit(X_train, y_train)

def print_accuracy(f):
    print(
        "Accuracy = {0}%".format(
            100 *
            np.mean(
                f(X_test) == y_test))
    )
    time.sleep(0.5)  # to let the print get out before any progress bars

print_accuracy(nn.predict)

Accuracy = 62.7%


In [6]:
import pickle

with open('../datasets/diabetes_brfss_dt.pkl', 'wb') as file:

    # A new file will be created
    pickle.dump(nn, file)

In [7]:
with open('../datasets/diabetes_brfss_train.csv', 'wb') as file:
    
        # A new file will be created
        X_train.to_csv(file, index=False)

In [8]:
with open('../datasets/diabetes_brfss_traintruth.csv', 'wb') as file:
    
    y_train.to_csv(file, index=False)